# Estimating parameter ranges

Before fitting a binary lens model, we need to establish physically realistic ranges for their parameters, notably the source angular radius $$\rho$$, the binary separation s and the binary mass ratio q.  Both spatial terms are given as ratios of the angular einstein radius.

In [8]:
from astropy import units as u
import astropy.constants as c
import numpy as np

The typical range of lens masses can be estimated from the mass range of main sequence stars

In [32]:
min_mass = 0.02*c.M_sun  # Brown dwarf
max_mass = 90.0*c.M_sun   # O star
mass_incr = (max_mass.value - min_mass.value)/ 10.0
ML = np.arange(min_mass.value, max_mass.value, mass_incr)  # Units: Kg
print(ML)

[3.97681974e+28 1.79314802e+31 3.58231922e+31 5.37149042e+31
 7.16066163e+31 8.94983283e+31 1.07390040e+32 1.25281752e+32
 1.43173464e+32 1.61065176e+32]


We adopt a typical lens distance as half the distance of the Sun from the Galactic center, as a reference point, and a source distance of twice that, which allows us to calculate a typical value for $$\mu = 1 + D_{L}/D_{LS}$$

In [31]:
DL = ((4000 * u.pc).to(u.m)).value  # Convert to m for consistency
DS = ((8000 * u.pc).to(u.m)).value
DLS = DS - DL
mu = 1.0 + DL/DLS

1.2342710325965468e+20 2.0


The angular Einstein radius can be calculated from $$\theta_E = \sqrt{\frac{4 G M_{L}}{\mu D_{L} c^2}}$$

In [39]:
thetaE = np.sqrt( (4.0 * c.G.value * ML)/ (mu * DL * c.c.value**2))
print('Range of the angular Einstein radius in radians: '+repr(thetaE))
thetaE_mas = thetaE * (180.0/np.pi) * 3600.0 * 1e6
print('Range of the angular Einstein radius in micro-arcsec: '+repr(thetaE_mas))

Range of the angular Einstein radius in radians: array([6.91767001e-10, 1.46892614e-08, 2.07622316e-08, 2.54237314e-08,
       2.93540794e-08, 3.28170357e-08, 3.59479303e-08, 3.88271754e-08,
       4.15071730e-08, 4.40243259e-08])
Range of the angular Einstein radius in micro-arcsec: array([ 142.6871865 , 3029.87765532, 4282.51766829, 5244.02102896,
       6054.71349654, 6768.99951974, 7414.79288121, 8008.67981906,
       8561.46899879, 9080.6690497 ])


## Source star angular radius

The parameter rho represents the angular size of the source star, normalized by the angular Einstein radius.  
This we can estimate for the typical range of stars expected, assuming the same source distance as above.

In [43]:
min_radius = ((0.01 * c.R_sun).to(u.m)).value
max_radius = ((100.0 * c.R_sun).to(u.m)).value  # In units of m
radius_incr = (max_radius - min_radius)/ 10.0
RS = np.arange(min_radius, max_radius, radius_incr)

We can calculate the angle subtended by this radius at distance D_S from basic trigonometry

In [45]:
thetaS = np.arctan(RS / DS)
print('Range of angular source radii in radians: ',thetaS)
thetaS_mas = thetaS * (180.0/np.pi) * 3600.0 * 1e6
print('Range of the angular source radius in micro-arcsec: '+repr(thetaS_mas))

Range of angular source radii in radians:  [2.81826269e-14 2.82079913e-11 5.63877999e-11 8.45676085e-11
 1.12747417e-10 1.40927226e-10 1.69107034e-10 1.97286843e-10
 2.25466652e-10 2.53646460e-10]
Range of the angular source radius in micro-arcsec: array([5.81308408e-03, 5.81831585e+00, 1.16308186e+01, 1.74433214e+01,
       2.32558242e+01, 2.90683269e+01, 3.48808297e+01, 4.06933325e+01,
       4.65058352e+01, 5.23183380e+01])


So we can now calculate the typical range of values for the rho parameter, taking a median value of thetaE

In [47]:
rho = thetaS / np.median(thetaE)
print('Range of values of dimensionless parameter, rho: ',rho)

Range of values of dimensionless parameter, rho:  [9.06614811e-07 9.07430764e-04 1.81395491e-03 2.72047906e-03
 3.62700321e-03 4.53352736e-03 5.44005151e-03 6.34657566e-03
 7.25309981e-03 8.15962396e-03]


## Binary separation

Microlensing is typically most sensitive to Galactic stellar binaries within the range 0 - 10 AU of the host star of the binary, bearing in mind that it's the projected angular separation of the binary components on sky at the time of the event that determines the morphology of the lightcurve.  To ensure we fully encompass this region, let's include 0 - 20 AU. 
This is incorporated into the model parameters as the quantity s, which is normalized by the angular Einstein radius.

So let's take a typical set of physical projected separations, aproj, for our binary, initially in AU:

In [63]:
aproj = np.arange(1e-3,20.0, 1.0)        # Avoid zero to bypass issues with logs later
aproj = ((aproj * u.au).to(u.m)).value   # Converting to m for consistency

Assuming a typical distance to the lens of 4Kpc as before, we can estimate the range of angular projected separations in radians from trigonometry.

In [64]:
s = np.arctan(aproj/DL)

But the model parameter normalizes all angular separations by the angular Einstein radius, so we calculate that radio:

In [65]:
s = s / np.median(thetaE)
s

array([3.89902674e-05, 3.90292577e-02, 7.80195251e-02, 1.17009793e-01,
       1.56000060e-01, 1.94990327e-01, 2.33980595e-01, 2.72970862e-01,
       3.11961130e-01, 3.50951397e-01, 3.89941665e-01, 4.28931932e-01,
       4.67922199e-01, 5.06912467e-01, 5.45902734e-01, 5.84893002e-01,
       6.23883269e-01, 6.62873537e-01, 7.01863804e-01, 7.40854072e-01])

Lastly, the binary separation parameter is usually fitted in logarithmic form:

In [66]:
logs = np.log10(s)
print('Range of log_10(s): ',logs)

Range of log_10(s):  [-4.40904379 -1.40860971 -1.1077967  -0.93177779 -0.80687523 -0.70998693
 -0.63082016 -0.56388371 -0.50589952 -0.45475302 -0.40900036 -0.36761162
 -0.32982635 -0.29506703 -0.26288473 -0.23292357 -0.20489666 -0.17856932
 -0.15374715 -0.13026733]


## Binary Mass Ratio

For stellar and planetary events, we want to consider the full possible range of masses in a two-component system where at least one component is a star.  

The primary object in a binary is usually considered to be the most massive of the two components.  So for this object, we can adopt the typical range of stellar masses:

In [70]:
min_m1 = (0.2 * c.M_sun).value
max_m1 = (90.0 * c.M_sun).value  # In Kg
m1_incr = (max_m1 - min_m1)/10.0
M1 = np.arange(min_m1, max_m1, m1_incr)
print('Range of masses for the primary binary object [Kg]: ',M1)

Range of masses for the primary binary object [Kg]:  [3.97681974e+29 1.82536026e+31 3.61095233e+31 5.39654439e+31
 7.18213645e+31 8.96772852e+31 1.07533206e+32 1.25389126e+32
 1.43245047e+32 1.61100968e+32]


The secondary, or smaller mass, component of a binary could be either a planet or a star, so the mass range is correspondingly wider:

In [86]:
min_m2 = (3e-6 * c.M_sun).value # Earth mass planet in Kg
max_m2 = (90.0 * c.M_sun).value  # In Kg
m2_incr = (max_m2 - min_m2)/100.0
M2 = np.arange(min_m2, max_m2, m2_incr)
print('Range of masses for the secondary binary object [Kg]: ',M2)

Range of masses for the secondary binary object [Kg]:  [5.96522961e+24 1.78957479e+30 3.57914361e+30 5.36871244e+30
 7.15828126e+30 8.94785009e+30 1.07374189e+31 1.25269877e+31
 1.43165566e+31 1.61061254e+31 1.78956942e+31 1.96852630e+31
 2.14748319e+31 2.32644007e+31 2.50539695e+31 2.68435383e+31
 2.86331071e+31 3.04226760e+31 3.22122448e+31 3.40018136e+31
 3.57913824e+31 3.75809513e+31 3.93705201e+31 4.11600889e+31
 4.29496577e+31 4.47392266e+31 4.65287954e+31 4.83183642e+31
 5.01079330e+31 5.18975019e+31 5.36870707e+31 5.54766395e+31
 5.72662083e+31 5.90557772e+31 6.08453460e+31 6.26349148e+31
 6.44244836e+31 6.62140525e+31 6.80036213e+31 6.97931901e+31
 7.15827589e+31 7.33723277e+31 7.51618966e+31 7.69514654e+31
 7.87410342e+31 8.05306030e+31 8.23201719e+31 8.41097407e+31
 8.58993095e+31 8.76888783e+31 8.94784472e+31 9.12680160e+31
 9.30575848e+31 9.48471536e+31 9.66367225e+31 9.84262913e+31
 1.00215860e+32 1.02005429e+32 1.03794998e+32 1.05584567e+32
 1.07374135e+32 1.09163704e+32

So now we can calculate a 2D grid of mass ratios of M2/M1

In [95]:
grid = np.meshgrid(M2,M1)
mass_ratios = grid[0]/grid[1]
print(mass_ratios)

[[1.00000000e+00 3.00000990e+05 6.00000980e+05 ... 2.91000000e+07
  2.94000000e+07 2.97000000e+07]
 [3.33332233e-06 1.00000000e+00 1.99999667e+00 ... 9.69996800e+01
  9.79996767e+01 9.89996733e+01]
 [1.66666394e-06 5.00000833e-01 1.00000000e+00 ... 4.84999208e+01
  4.89999200e+01 4.94999192e+01]
 ...
 [3.43642611e-08 1.03093124e-02 2.06185904e-02 ... 1.00000000e+00
  1.01030928e+00 1.02061856e+00]
 [3.40136054e-08 1.02041153e-02 2.04081966e-02 ... 9.89795919e-01
  1.00000000e+00 1.01020408e+00]
 [3.36700337e-08 1.01010434e-02 2.02020532e-02 ... 9.79797980e-01
  9.89898990e-01 1.00000000e+00]]


However, this grid includes some cases where M1 < M2, meaning the ratio is >1, which is contrary to our definition above.  So we eliminate those values to reach the final range of expected mass_ratio values.  

In [96]:
mask = (mass_ratios > 1.0)
q = np.ma.MaskedArray(mass_ratios, mask=mask)
q

masked_array(
  data=[[1.0, --, --, ..., --, --, --],
        [3.3333223333696337e-06, 1.0, --, ..., --, --, --],
        [1.666663944448891e-06, 0.5000008333319722, 1.0, ..., --, --, --],
        ...,
        [3.436426113295781e-08, 0.010309312360505862,
         0.020618590356750593, ..., 1.0, --, --],
        [3.401360541903836e-08, 0.010204115299180872,
         0.020408196584756327, ..., 0.9897959187144244, 1.0, --],
        [3.367003365869696e-08, 0.010101043430942409,
         0.02020205319185116, ..., 0.9797979804781826,
         0.9898989902390913, 1.0]],
  mask=[[False,  True,  True, ...,  True,  True,  True],
        [False, False,  True, ...,  True,  True,  True],
        [False, False, False, ...,  True,  True,  True],
        ...,
        [False, False, False, ..., False,  True,  True],
        [False, False, False, ..., False, False,  True],
        [False, False, False, ..., False, False, False]],
  fill_value=1e+20)

Lastly, the q model parameter is typically fitted as a log_10 value:

In [99]:
logq = np.log10(q)
print('Range of log10(q) values: ',logq.max(),' - ',logq.min())

Range of log10(q) values:  0.0  -  -7.472756449463439
